In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, rdMolDescriptors
import numpy as np
from rdkit.Chem.Draw.IPythonConsole import molSize
from tqdm import tqdm

In [ ]:
import pandas as pd

def validate_smiles(smiles):

    if pd.isna(smiles) or smiles.strip().lower() == "nan":
        return "SMILES is 'nan'"
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return "SMILES false"
        return None
    except Exception as e:
        return f"error: {str(e)}"


df = pd.read_csv('data/1-smiles/train_set.csv', names=["label", "id", "smiles"])


errors = []
for idx, row in df.iterrows():
    smiles = row["smiles"]
    error = validate_smiles(smiles)
    if error:
        errors.append({
            "index": idx + 1,
            "ID": row["id"],
            "SMILES": smiles,
            "error": error
        })

if errors:
    error_df = pd.DataFrame(errors)
    print(error_df.to_string(index=False))
else:
    print("success！")

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit import rdBase

rdBase.DisableLog('rdApp.warning')

def get_all_descriptors(mol):

    if mol is None:
        return None

    descriptor_names = [x[0] for x in Descriptors._descList]
    calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

    try:

        descriptors = calculator.CalcDescriptors(mol)
        return dict(zip(descriptor_names, descriptors))
    except:
        return None

def process_smiles_pair(smiles):
    if pd.isna(smiles):
        return None


    mol_smiles = smiles.split('.')
    if len(mol_smiles) != 2:
        return None

    mols = [Chem.MolFromSmiles(s) for s in mol_smiles]
    descriptors = [get_all_descriptors(mol) for mol in mols]

    if None in descriptors:
        return None


    combined = {}
    for key in descriptors[0].keys():
        combined[f'mol1_{key}'] = descriptors[0][key]
        combined[f'mol2_{key}'] = descriptors[1][key]

        combined[f'avg_{key}'] = (descriptors[0][key] + descriptors[1][key])/2
        combined[f'diff_{key}'] = abs(descriptors[0][key] - descriptors[1][key])

    return combined

def main():

    df = pd.read_csv('data/1-smiles/train_set.csv')


    results = []
    for _, row in df.iterrows():
        descriptors = process_smiles_pair(row['SMILES'])
        if descriptors is not None:
            descriptors['label'] = row['label']
            descriptors['ID'] = row['ID']
            descriptors['SMILES'] = row['SMILES']
            results.append(descriptors)


    results_df = pd.DataFrame(results)


    results_df.to_csv('data/2-descriptor/all_descriptors.csv', index=False)

    if len(results) > 0:
        descriptor_keys = [key for key in results[0].keys()
                         if key not in ['label', 'ID', 'SMILES']]


        descriptor_groups = {
            'basic': ['mol1_MolWt', 'mol2_MolWt', 'mol1_MolLogP', 'mol2_MolLogP'],
            'topological': [key for key in descriptor_keys if 'Chi' in key or 'Kappa' in key],
            'connectivity': [key for key in descriptor_keys if 'BertzCT' in key or 'Ipc' in key],
            'surface': [key for key in descriptor_keys if 'TPSA' in key or 'LabuteASA' in key]
        }


        for group_name, group_keys in descriptor_groups.items():
            cols = ['label', 'ID', 'SMILES'] + group_keys
            try:
                group_df = results_df[cols]
                group_df.to_csv(f'data/2-descriptor/{group_name}_descriptors.csv', index=False)
            except KeyError as e:
                print(f"e")

main()

In [ ]:
df = pd.read_csv('data/2-descriptor/all_descriptors.csv')
df